#### importação das bibliotecas

In [1]:
import sys
sys.path.append('../')

import pandas as pd
#pd.set_option('display.float_format',lambda x: f'%.0f')

#### carregando arquivo de análise

In [2]:
df = pd.read_csv('../data/raw/consumo_energia_eletrica.csv',sep= ',')

### Análise Descritiva

#### Análise duplicadas 

***

In [3]:
df.duplicated().sum()

np.int64(1017)

In [4]:
#def count_duplicates(dataframe: pd.DataFrame):
#    return dataframe.duplicated().sum()

In [5]:
#def show_data (dataframe: pd.DataFrame):
#    return dataframe.sample(10)

In [6]:
from src.data_cleaning.duplicate_data import count_duplicates, check_duplicates

In [7]:
import src.data_cleaning.duplicate_data as dd

In [8]:
dd.count_duplicates(df)

Foram encontrados 1017 de registros duplicados


np.int64(1017)

In [9]:
dd.check_duplicates(df)

Lista com os dados duplicados


0        False
1        False
2        False
3        False
4        False
         ...  
39892     True
39893     True
39894     True
39895     True
39896     True
Length: 39897, dtype: bool

In [10]:
df_sem_duplicados = dd.drop_duplicates(df)
check_duplicates(df_sem_duplicados)

Lista com os dados duplicados


0        False
1        False
2        False
3        False
4        False
         ...  
38875    False
38876    False
38877    False
38878    False
38879    False
Length: 38880, dtype: bool

#### Análise de dados ausentes

***

In [11]:
df_sem_duplicados.isna().sum()

ano                        0
mes                        0
sigla_uf                   0
tipo_consumo               0
numero_consumidores    12960
consumo                    0
dtype: int64

In [12]:
dados_ausentes = df_sem_duplicados.isna().sum()
total_registros = len(df_sem_duplicados)

In [13]:
total_registros

38880

In [14]:
# dataframe com os resultados
ausentes = dados_ausentes.reset_index(name = 'qtd')
ausentes.columns = ['coluna','quantidade'] #Renomeia colunas
ausentes

,coluna,quantidade
0,ano,0
1,mes,0
2,sigla_uf,0
3,tipo_consumo,0
4,numero_consumidores,12960
5,consumo,0


In [15]:
#### Calcula percentual do total
ausentes['percentual'] = ausentes['quantidade'] / total_registros * 100

In [16]:
ausentes['percentual'] = ausentes['percentual'].apply(lambda x: f'{x: .1f}%')

In [17]:
filtro = ausentes['quantidade']>0
ausentes[filtro]

,coluna,quantidade,percentual
4,numero_consumidores,12960,33.3%


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39897 entries, 0 to 39896
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  39897 non-null  int64  
 1   mes                  39897 non-null  int64  
 2   sigla_uf             39897 non-null  object 
 3   tipo_consumo         39897 non-null  object 
 4   numero_consumidores  26937 non-null  float64
 5   consumo              39897 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.8+ MB


In [19]:
df['ano'] = df['ano'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39897 entries, 0 to 39896
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  39897 non-null  float64
 1   mes                  39897 non-null  int64  
 2   sigla_uf             39897 non-null  object 
 3   tipo_consumo         39897 non-null  object 
 4   numero_consumidores  26937 non-null  float64
 5   consumo              39897 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 1.8+ MB


In [20]:
filtro = df_sem_duplicados['numero_consumidores'].isna()
df_sem_duplicados.loc[filtro,:]

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
0,2004,1,TO,Total,NaN,65876
1,2004,1,BA,Total,NaN,1444451
2,2004,1,PR,Total,NaN,1596274
3,2004,1,RS,Total,NaN,1780912
4,2004,1,GO,Total,NaN,630624
...,...,...,...,...,...,...
37579,2023,12,DF,Cativo,NaN,503565
37580,2023,12,MT,Cativo,NaN,732101
37581,2023,12,RR,Cativo,NaN,112337
37582,2023,12,CE,Cativo,NaN,887204


In [21]:
df_sem_duplicados.groupby('tipo_consumo')['numero_consumidores'].mean().reset_index()

,tipo_consumo,numero_consumidores
0,Cativo,NaN
1,Comercial,1.950837e+05
2,Industrial,1.933195e+04
3,Outros,1.713510e+05
4,Residencial,2.343447e+06
5,Total,NaN


In [22]:
filtro = (df_sem_duplicados['sigla_uf'] == 'MG') & (df_sem_duplicados['ano'] == 2023) & (df_sem_duplicados['mes'] == 1)
df_sem_duplicados[filtro]

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
36951,2023,1,MG,Total,NaN,5052956
37262,2023,1,MG,Cativo,NaN,2285706
37601,2023,1,MG,Outros,627736.0,498609
37917,2023,1,MG,Comercial,995468.0,689840
38255,2023,1,MG,Industrial,34307.0,2731812
38564,2023,1,MG,Residencial,8045200.0,1132693
